<a href="https://colab.research.google.com/github/ashioyajotham/Daily-ML/blob/main/One_for_the_World_Product_Engineer_Trial_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# One for the World Product Engineer Trial Task

In this trial task you will undertake a typical data processing assignment. We have a simple database, containing three tables - donations, impact, and portfolio

Donations has the following columns:

`donor_id`, `donation_amount`, `donation_currency`, `portfolio`

Donors make donations to 'portfolios' which are comprised of one or more charities. For example, most donors donate to 'One for the World Top Picks', which splits their donation equally to GiveWell's top-rated charities (Against Malaria Foundation, New Incentives, Malaria Consortium, Helen Keller International).

If a donor wants to donate to just one charity, they technically donate to a 'portfolio' which has a 100% allocation to that charity.

This is stored in the 'portfolio' table, which has the columns:

`portfolio_name`, `charity_name`, `allocation_proportion`

Every quarter, we send impact reports to our donors, where we convert what they have donated into tangible outcomes, such as the number of people protected against malaria or vaccine immunizations incentivized.

The cost per 'intervention' is stored in the 'impact' table, which has the columns:

`charity_name`, `intervention`, `usd_per_intervention`

Your task is to produce a result where each donor_id has their individual impacts listed, ready to load into our newsletter tool.

For example, if the donations table had the following rows:

donor_id | donation_amount | donation_currency | portfolio
--- | --- | --- | ---
1 | 10 | USD | Malaria Portfolio
2 | 20 | USD | New Incentives

And the portfolio table had the following rows (note how the Malaria Portfolio is split 75%/25% across two charities, and the New Incentives 'portfolio' is allocated 100% to New Incentives):

portfolio_name | charity_name | allocation_proportion
--- | --- | ---
Malaria Portfolio | Against Malaria Foundation | 0.75
Malaria Portfolio | Malaria Consortium | 0.25
New Incentives | New Incentives | 1

And the impact table had the following rows:

charity_name | intervention | usd_per_intervention
--- | --- | ---
Against Malaria Foundation | people protected against malaria	| 0.5
Malaria Consortium | children given anti-malarial treatments	 | 0.3
New Incentives | vaccine immunizations incentivized	 | 1.4

Then the table you would produce would look like:

donor_id | impact_1 | impact_2 | impact_3 | impact_4
--- | --- | --- | --- | ---
1 | 15 people protected against malaria | 8 children given anti-malarial treatments
2 | 14 vaccine immunizations incentivized

The impacts have been calculated by taking the amount donated to each charity, and the interventions that funds. In this example, the donor with id 1 has donated 10 USD to the Malaria Portfolio, which is split \$7.50 to the Against Malaria Foundation and \$2.50 to the Malaria Consortium. The Against Malaria Foundation's interventions cost \$0.50, so \$7.50 protects 15 people against malaria. The Malaria Consortium's interventions cost \$0.30, so $2.50 buys 14 anti-malarial treatments.

## Important things to note
- The output should:
  - Have one row per donor
  - Fill in impacts from `impact_1`, then `impact_2` etc. In other words if a donor has donated to only one charity, that impact should be in the `impact_1` column. `impact_1` should not be empty if `impact_2`/`impact_3`/`impact_4` are filled
  - Round down the number of interventions to the nearest whole number. In the above example, donor 2 donated \$20 to New Incentives which naively produces 14.28 interventions, but this is rounded down to 14 in the final output
- Only use pure SQL. In other words not SQLX or anything else that compiles to SQL, nor python/pandas. Your output should be a single SQL 'query' (it may use CTEs and subqueries, but should fundamentally be a single string) that produces a report in the format outlined above
- You may have to do some data cleaning and normalizing, use any sensible exchange rate to do currency conversions
- Do not use AI-assistance (ie. large language models such as ChatGPT), though you may use the internet

## How to complete the task
- You can either make a copy of this notebook, and use Google Colab (File -> Save a copy in Drive)
- Or download the .ipynb file (File -> Download -> Download .ipynb) and use local notebook software (eg. [Jupyter](https://jupyter.org/) or [VS Code](https://code.visualstudio.com/docs/datascience/jupyter-notebooks), or hosted notebook software (eg. [Deepnote](https://deepnote.com/))

The following cell downloads the database to your file system. You don't need to understand how it works, so long as it runs without errors.

If you are having issues, you can download the file directly from the link stored in the `db_url` variable.

In [1]:
import sqlalchemy
import requests
import os

# --- Configuration ---
db_url = "https://storage.googleapis.com/oftw-public/oftw_database.db"
local_db_filename = "oftw_database_downloaded.db" # Use a distinct local name

print(f"Downloading database from {db_url} to {local_db_filename}...")
try:
    response = requests.get(db_url)
    response.raise_for_status() # Check for download errors (like 404 Not Found)

    with open(local_db_filename, 'wb') as f:
        f.write(response.content) # Since it's small, read all content at once
    print(f"Successfully downloaded.")

except requests.exceptions.RequestException as e:
    print(f"Error downloading database file: {e}")
    raise # Stop execution if download fails

# Ensure the file exists locally after download attempt
if os.path.exists(local_db_filename):
    # CORRECT connection string: points to the LOCAL file
    connection_string = f"sqlite:///{local_db_filename}"
    print(f"Connecting to local database: {connection_string}")

    try:
        # Create the SQLAlchemy engine for the LOCAL file
        engine = sqlalchemy.create_engine(connection_string)

        # --- Now you can use the engine ---
        print("Successfully created SQLAlchemy engine.")

        # Example: List tables
        with engine.connect() as connection:
            inspector = sqlalchemy.inspect(engine)
            table_names = inspector.get_table_names()
            print(f"Tables found in the database: {table_names}")

    except Exception as e:
        print(f"Error creating SQLAlchemy engine or interacting with DB: {e}")

else:
    print(f"Local database file '{local_db_filename}' not found after download attempt.")

Successfully downloaded.
Connecting to local database: sqlite:///oftw_database_downloaded.db
Successfully created SQLAlchemy engine.
Tables found in the database: ['donations', 'impact', 'portfolio']


The following code cells show how to write queries using sqlalchemy to display the rows in the 3 tables

In [2]:
connection = engine.connect()
select_impact_query = sqlalchemy.text(f"SELECT * FROM impact")
result_all = connection.execute(select_impact_query)
result_all.fetchall()

[('Helen Keller International', "children given a year's Vitamin A supplementation", 2.0),
 ('Malaria Consortium', 'children given anti-malarial treatments', 7.0),
 ('Against Malaria Foundation', 'people protected against malaria', 2.78),
 ('New Incentives', 'vaccine immunizations incentivized', 0.6)]

In [3]:
select_donations_query = sqlalchemy.text(f"SELECT * FROM donations")
result_all = connection.execute(select_donations_query)
result_all.fetchall()

[('4e1363e1-0eda-4f90-a1f5-ca4d1d9fc146', 100.0, 'USD', 'OFTW Top Picks'),
 ('4e1363e1-0eda-4f90-a1f5-ca4d1d9fc146', 200.0, 'USD', 'OFTW Top Picks'),
 ('4e1363e1-0eda-4f90-a1f5-ca4d1d9fc146', 300.0, 'USD', 'OFTW Top picks'),
 ('4e1363e1-0eda-4f90-a1f5-ca4d1d9fc146', 400.0, 'USD', 'OFTW top picks'),
 ('bced85e7-ee9c-45de-8ef8-3dca450ad98d', 50.0, 'CAD', 'Against Malaria Foundation'),
 ('bced85e7-ee9c-45de-8ef8-3dca450ad98d', 50.0, 'CAD', 'Against Malaria Foundation'),
 ('bced85e7-ee9c-45de-8ef8-3dca450ad98d', 50.0, 'cad', 'Against Malaria foundation'),
 ('cadde239-a711-4598-bc50-12eebb4afaac', 100.5, 'gbp', 'OFTW Top Picks'),
 ('cadde239-a711-4598-bc50-12eebb4afaac', 200.0, 'GBP', 'OFTW Top Picks'),
 ('cadde239-a711-4598-bc50-12eebb4afaac', 100.5, 'GBP', 'OFTW top picks'),
 ('cadde239-a711-4598-bc50-12eebb4afaac', 50.75, 'GBP', 'OFTW Top Picks'),
 ('4cedb12e-6c28-46b6-a77a-2814fc9a79f0', 5000.0, 'AuD', 'New Incentives'),
 ('4cedb12e-6c28-46b6-a77a-2814fc9a79f0', 10000.0, 'AUD', 'New Inc

In [4]:
select_portfolio_query = sqlalchemy.text(f"SELECT * FROM portfolio")
result_all = connection.execute(select_portfolio_query)
result_all.fetchall()

[('OFTW Top Picks', 'Helen Keller International', 0.25),
 ('OFTW Top Picks', 'Malaria Consortium', 0.25),
 ('OFTW Top Picks', 'Against Malaria Foundation', 0.25),
 ('OFTW Top Picks', 'New Incentives', 0.25),
 ('Against Malaria Foundation', 'Against Malaria Foundation', 1.0),
 ('New Incentives', 'New Incentives', 1.0)]

Fill in your answer in the cell below

In [5]:
# Put your query as the 'query' variable
query = """
WITH 
-- First, we normalize portfolio names (case insensitive matching)
normalized_donations AS (
    SELECT 
        donor_id,
        donation_amount,
        CASE 
            WHEN UPPER(donation_currency) = 'USD' THEN donation_amount
            WHEN UPPER(donation_currency) = 'GBP' THEN donation_amount * 1.25 -- GBP to USD conversion
            WHEN UPPER(donation_currency) = 'CAD' THEN donation_amount * 0.75 -- CAD to USD conversion
            WHEN UPPER(donation_currency) = 'AUD' THEN donation_amount * 0.65 -- AUD to USD conversion
            ELSE donation_amount -- Default case
        END AS donation_amount_usd,
        CASE 
            WHEN UPPER(portfolio) = 'OFTW TOP PICKS' THEN 'OFTW Top Picks'
            WHEN UPPER(portfolio) = 'AGAINST MALARIA FOUNDATION' THEN 'Against Malaria Foundation'
            WHEN UPPER(portfolio) = 'NEW INCENTIVES' THEN 'New Incentives'
            ELSE portfolio
        END AS normalized_portfolio
    FROM donations
),

-- Then we calculate the donation amount to each charity based on portfolio allocation
donations_per_charity AS (
    SELECT 
        d.donor_id,
        p.charity_name,
        SUM(d.donation_amount_usd * p.allocation_proportion) AS amount_to_charity
    FROM normalized_donations d
    JOIN portfolio p ON d.normalized_portfolio = p.portfolio_name
    GROUP BY d.donor_id, p.charity_name
),

-- We will then calculate impacts for each donor and charity
impacts AS (
    SELECT 
        d.donor_id,
        d.charity_name,
        i.intervention,
        FLOOR(d.amount_to_charity / i.usd_per_intervention) AS intervention_count,
        ROW_NUMBER() OVER (PARTITION BY d.donor_id ORDER BY d.amount_to_charity DESC) AS impact_number
    FROM donations_per_charity d
    JOIN impact i ON d.charity_name = i.charity_name
    WHERE d.amount_to_charity > 0
),

-- Next, we prepare impact strings
impact_strings AS (
    SELECT 
        donor_id,
        impact_number,
        intervention_count || ' ' || intervention AS impact_string
    FROM impacts
    WHERE intervention_count > 0
)

-- Final result with pivoted impacts
SELECT 
    donor_id,
    MAX(CASE WHEN impact_number = 1 THEN impact_string ELSE NULL END) AS impact_1,
    MAX(CASE WHEN impact_number = 2 THEN impact_string ELSE NULL END) AS impact_2,
    MAX(CASE WHEN impact_number = 3 THEN impact_string ELSE NULL END) AS impact_3,
    MAX(CASE WHEN impact_number = 4 THEN impact_string ELSE NULL END) AS impact_4
FROM impact_strings
GROUP BY donor_id
ORDER BY donor_id
"""

# Execute query and fetch results
result = connection.execute(sqlalchemy.text(query))
all_results = result.fetchall()

In [6]:
# Let's checkout the results to see if they match our expectations
import pandas as pd
df = pd.DataFrame(all_results, columns=['donor_id', 'impact_1', 'impact_2', 'impact_3', 'impact_4'])
    
print("DONOR IMPACT REPORT:")
display(df)

DONOR IMPACT REPORT:


,donor_id,impact_1,impact_2,impact_3,impact_4
0,4cedb12e-6c28-46b6-a77a-2814fc9a79f0,27083 vaccine immunizations incentivized,None,None,None
1,4e1363e1-0eda-4f90-a1f5-ca4d1d9fc146,125 children given a year's Vitamin A suppleme...,35 children given anti-malarial treatments,89 people protected against malaria,416 vaccine immunizations incentivized
2,7224656a-5552-4605-80ef-160b316e3a20,527 children given a year's Vitamin A suppleme...,150 children given anti-malarial treatments,379 people protected against malaria,1758 vaccine immunizations incentivized
3,bced85e7-ee9c-45de-8ef8-3dca450ad98d,40 people protected against malaria,None,None,None
4,cadde239-a711-4598-bc50-12eebb4afaac,70 children given a year's Vitamin A supplemen...,20 children given anti-malarial treatments,50 people protected against malaria,235 vaccine immunizations incentivized
